# RaLSGAN (Relativistic Average Least Squares GAN) Code Guide
This notebook implements **RaLSGAN** using **TensorFlow/Keras** on the MNIST dataset. The code is modular and serves as a reference for future projects.

## Step 1: Install Dependencies and Import Libraries

In [1]:

# Install TensorFlow if not already installed
# !pip install tensorflow

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


## Step 2: Load and Preprocess the Dataset

In [14]:

# Load and preprocess the MNIST dataset
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()

# Normalize images to the range [-1, 1]
train_images = train_images.reshape(-1, 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5\



# Create a TensorFlow dataset
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(60000).batch(batch_size)


In [15]:
print("Generator Gradients:", gen_gradients)
print("Discriminator Gradients:", disc_gradients)


NameError: name 'gen_gradients' is not defined

## Step 3: Define the Generator and Discriminator Models

In [3]:

# Generator
def build_generator(z_dim):
    model = tf.keras.Sequential([
        layers.Dense(256, input_dim=z_dim),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(1024),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(28 * 28, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ])
    return model

# Discriminator
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(1024),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(1)  # Single scalar output
    ])
    return model


## Step 4: Define Relativistic Loss Functions

In [4]:

# Relativistic Discriminator Loss
def discriminator_loss(real_output, fake_output):
    real_loss = tf.reduce_mean(tf.square(real_output - tf.reduce_mean(fake_output) - 1))
    fake_loss = tf.reduce_mean(tf.square(fake_output - tf.reduce_mean(real_output) + 1))
    return real_loss + fake_loss

# Relativistic Generator Loss
def generator_loss(real_output, fake_output):
    real_loss = tf.reduce_mean(tf.square(real_output - tf.reduce_mean(fake_output) + 1))
    fake_loss = tf.reduce_mean(tf.square(fake_output - tf.reduce_mean(real_output) - 1))
    return real_loss + fake_loss


## Step 5: Training Setup

In [5]:

# Hyperparameters
z_dim = 100  # Latent space dimension
epochs = 50
learning_rate = 0.0002

# Instantiate Generator and Discriminator
generator = build_generator(z_dim)
discriminator = build_discriminator()

# Optimizers
gen_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5)
disc_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5)


c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Step 6: Define Training Loop

In [18]:
@tf.function
def train_step(real_images):
    batch_size = tf.shape(real_images)[0]
    
    # Generate random noise and fake images
    random_noise = tf.random.normal([batch_size, z_dim])
    fake_images = generator(random_noise)
    
    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        # Compute Discriminator outputs
        real_output = discriminator(real_images)
        fake_output = discriminator(fake_images)
        
        # Compute losses
        d_loss = discriminator_loss(real_output, fake_output)
        g_loss = generator_loss(real_output, fake_output)
    
    # Compute gradients
    disc_gradients = disc_tape.gradient(d_loss, discriminator.trainable_variables)
    gen_gradients = gen_tape.gradient(g_loss, generator.trainable_variables)
    
    # Debug: Check if gradients are None
    print("Discriminator Gradients:", disc_gradients)
    print("Generator Gradients:", gen_gradients)
    
    # Update weights
    disc_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
    gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    
    return d_loss, g_loss


In [19]:

# @tf.function
# def train_step(real_images):
#     batch_size = tf.shape(real_images)[0]
    
#     # Generate random noise and fake images
#     random_noise = tf.random.normal([batch_size, z_dim])
#     fake_images = generator(random_noise)
    
#     with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
#         # Compute Discriminator outputs
#         real_output = discriminator(real_images)
#         fake_output = discriminator(fake_images)
        
#         # Compute losses
#         d_loss = discriminator_loss(real_output, fake_output)
#         g_loss = generator_loss(real_output, fake_output)
    
#     # Compute gradients and update weights
#     disc_gradients = disc_tape.gradient(d_loss, discriminator.trainable_variables)
#     gen_gradients = gen_tape.gradient(g_loss, generator.trainable_variables)
    
#     disc_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
#     gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    
#     return d_loss, g_loss

# # Full training loop
# def train(dataset, epochs):
#     for epoch in range(epochs):
#         for batch in dataset:
#             d_loss, g_loss = train_step(batch)
        
#         # Print losses after each epoch
#         print(f"Epoch {epoch + 1}/{epochs}, D Loss: {d_loss.numpy()}, G Loss: {g_loss.numpy()}")
        
#         # Generate and visualize images after every epoch
#         if (epoch + 1) % 10 == 0:
#             generate_and_save_images(generator, epoch + 1, z_dim)


## Step 7: Generate and Save Images

In [20]:

def generate_and_save_images(model, epoch, z_dim, num_images=16):
    random_noise = tf.random.normal([num_images, z_dim])
    fake_images = model(random_noise).numpy()
    fake_images = (fake_images + 1) / 2.0  # Rescale to [0, 1]
    
    # Plot the generated images
    fig, axes = plt.subplots(1, num_images, figsize=(20, 2))
    for i in range(num_images):
        axes[i].imshow(fake_images[i, :, :, 0], cmap='gray')
        axes[i].axis('off')
    plt.savefig(f"epoch_{epoch}.png")
    plt.show()

# Start training
train(dataset, epochs)


Discriminator Gradients: [<tf.Tensor 'AddN_2:0' shape=(784, 1024) dtype=float32>, <tf.Tensor 'AddN_3:0' shape=(1024,) dtype=float32>, <tf.Tensor 'AddN_4:0' shape=(1024, 512) dtype=float32>, <tf.Tensor 'AddN_5:0' shape=(512,) dtype=float32>, <tf.Tensor 'AddN_6:0' shape=(512, 256) dtype=float32>, <tf.Tensor 'AddN_7:0' shape=(256,) dtype=float32>, <tf.Tensor 'AddN_8:0' shape=(256, 1) dtype=float32>, <tf.Tensor 'AddN_9:0' shape=(1,) dtype=float32>]
Generator Gradients: [None, None, None, None, None, None, None, None]


ValueError: in user code:

    File "C:\Users\Rishu\AppData\Local\Temp\ipykernel_19068\1914475880.py", line 28, in train_step  *
        gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    File "c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\optimizers\base_optimizer.py", line 344, in apply_gradients  **
        self.apply(grads, trainable_variables)
    File "c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\optimizers\base_optimizer.py", line 397, in apply
        grads, trainable_variables = self._filter_empty_gradients(
    File "c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\optimizers\base_optimizer.py", line 729, in _filter_empty_gradients
        raise ValueError("No gradients provided for any variable.")

    ValueError: No gradients provided for any variable.


In [4]:
import numpy as np
import zipfile
import os
from time import time
import matplotlib.pyplot as plt
from scipy.stats import truncnorm

import read_data_dogimage

import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Reshape, Flatten, concatenate, UpSampling2D, Conv2DTranspose, BatchNormalization, LeakyReLU, GlobalAveragePooling2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD, Adam
from keras import optimizers
from keras import backend as K
from PIL import Image

if 'tensorflow' == K.backend():
    import tensorflow as tf
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

batch_size = 32
lr = 0.0005
beta1 = 0.5
epochs = 30

nz = 256


def convtlayer(input, filter, kernel_size, stride, padding):
    x = Conv2DTranspose(filters=filter, kernel_size=kernel_size, strides=stride, padding=padding, use_bias=False, kernel_initializer='glorot_uniform')(input)
    x = BatchNormalization(momentum=0.9, epsilon=1e-05)(x)
    x = Activation(activation='relu')(x)
    return x


def generator():
    input = Input(shape=(nz, ))
    x = Reshape((1, 1, nz))(input)
    x = convtlayer(x, 1024, 4, 1, 'valid')  # as FC layer
    x = convtlayer(x, 512, 4, 2, 'same')
    x = convtlayer(x, 256, 4, 2, 'same')
    x = convtlayer(x, 128, 4, 2, 'same')
    x = convtlayer(x, 64, 4, 2, 'same')
    x = Conv2DTranspose(filters=3, kernel_size=3, strides=1, padding='same', use_bias=False, kernel_initializer='glorot_uniform')(x)
    x = Activation(activation='tanh')(x)

    gene = Model(input, x, name="generator")
    gene.summary()

    return gene


def convlayer(input, filter, kernel_size, stride, padding, bn=False):
    x = Conv2D(filters=filter, kernel_size=kernel_size, strides=stride, padding=padding, use_bias=False, kernel_initializer='glorot_uniform')(input)
    if bn:
        x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x


def discriminator():
    input = Input(shape=(64, 64, 3))
    x = convlayer(input, 32, 4, 2, 'same')
    x = convlayer(x, 64, 4, 2, 'same')
    x = convlayer(x, 128, 4, 2, 'same', True)
    x = convlayer(x, 256, 4, 2, 'same', True)
    x = Conv2D(filters=1, kernel_size=4, strides=1, padding='valid', use_bias=False)(x)  # as FC layer

    disc = Model(input, x, name="discriminator")
    disc.summary()

    return disc


def train():
    gene = generator()
    disc = discriminator()

    real_img = Input(shape=(64, 64, 3))
    noise_input = Input(shape=(nz, ))

    # Build Generator Network
    fake_img = gene(noise_input)
    # Build 2 Discriminator Networks (one from noise input, one from generated samples)
    disc_real = disc(real_img)  # C(x_r)
    disc_fake = disc(fake_img)  # C(x_f)

    # Build Loss
    disc_real_average = K.mean(disc_real, axis=0)
    disc_fake_average = K.mean(disc_fake, axis=0)

    def lossD(y_true, y_pred):
        # epsilon=0.000001
        # return -(K.mean(K.log(K.sigmoid(disc_real_average - disc_fake_average) + epsilon), axis=0) + K.mean(K.log(1 -
        # K.sigmoid(disc_fake_average - disc_real_average) + epsilon), axis=0))
        return K.mean(K.pow(disc_real_average - disc_fake_average - 1, 2), axis=0) + K.mean(K.pow(disc_fake_average - disc_real_average + 1, 2), axis=0)

    def lossG(y_true, y_pred):
        # epsilon=0.000001
        # return -(K.mean(K.log(K.sigmoid(disc_fake_average - disc_real_average) + epsilon), axis=0) + K.mean(K.log(1 -
        # K.sigmoid(disc_real_average - disc_fake_average) + epsilon), axis=0))
        return K.mean(K.pow(disc_fake_average - disc_real_average - 1, 2), axis=0) + K.mean(K.pow(disc_real_average - disc_fake_average + 1, 2), axis=0)

    # Build Optimizers
    adamOP = Adam(lr=lr, beta_1=beta1)

    # Build trainable generator and discriminator
    disc_train = Model([noise_input, real_img], [disc_real, disc_fake])
    gene.trainable = False
    disc.trainable = True
    disc_train.compile(optimizer=adamOP, loss=[lossD, None])
    disc_train.summary()

    gene_train = Model([noise_input, real_img], [disc_real, disc_fake])
    gene.trainable = True
    disc.trainable = False
    gene_train.compile(optimizer=adamOP, loss=[lossG, None])
    gene_train.summary()

    # Start training
    ideal_target = np.zeros((batch_size, 1), dtype=np.float32)

    # Load data
    images, names = read_data_dogimage.load_data()
    images = images / 255 * 2 - 1

    batch_num = int(len(images[0:]) // (batch_size * 2))

    loss_d = []
    loss_g = []

    for epoch in np.arange(epochs):
        np.random.shuffle(images)

        print('current step: {:d} / {:d}, {:.2f}'.format((epoch + 1), epochs, (epoch + 1)/epochs))
        start_time = time()

        for batch_i in np.arange(0, batch_num):
            batch = images[batch_i * (batch_size * 2): (batch_i + 1) * (batch_size * 2)]

            # The result may be affected by the order or the frequency of training gene or disc per epoch.

            batch_sec = batch[0 * batch_size: 1 * batch_size]
            # noise = np.random.randn(batch_size, nz).astype(np.float32)
            noise = truncnorm.rvs(-1.0, 1.0, size=(batch_size, nz)).astype(np.float32)
            gene.trainable = True
            disc.trainable = False
            loss_g.append(gene_train.train_on_batch([noise, batch_sec], ideal_target))

            batch_sec = batch[1 * batch_size: 2 * batch_size]
            # noise = np.random.randn(batch_size, nz).astype(np.float32)
            noise = truncnorm.rvs(-1.0, 1.0, size=(batch_size, nz)).astype(np.float32)
            gene.trainable = False
            disc.trainable = True
            loss_d.append(disc_train.train_on_batch([noise, batch_sec], ideal_target))

        print('lossG: {}, lossD: {}'.format(loss_g[-1][0], loss_d[-1][0]))
        print('epoch time: {}\n'.format(time() - start_time))

    plt.plot(np.array(loss_g)[:, 0])
    plt.plot(np.array(loss_d)[:, 0])
    plt.legend(['generator', 'discriminator'])
    plt.savefig('loss.png')

    return gene


class ImageGenerator:
    act = 0

    def __init__(self, gene):
        self.gene = gene

    def get_fake_img(self):
        # noise = np.random.randn(1, nz).astype(np.float32)
        noise = truncnorm.rvs(-1.0, 1.0, size=(1, nz)).astype(np.float32)
        img = ((self.gene.predict(noise)[0].reshape((64, 64, 3)) + 1) / 2) * 255
        self.act = (self.act + 1) % 10000
        return Image.fromarray(img.astype('uint8'))


if __name__ == '__main__':
    gene = train()
    I = ImageGenerator(gene)

    z = zipfile.PyZipFile('images.zip', mode='w')
    for k in range(10):
        img = I.get_fake_img()
        f = str(k) + '.png'
        img.save(f, 'PNG')
        z.write(f)
        os.remove(f)
        if k % 1000 == 0:
            print(k)
    z.close()
    print('completed')


ModuleNotFoundError: No module named 'read_data_dogimage'

In [3]:
pip install dogimage

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement dogimage (from versions: none)
ERROR: No matching distribution found for dogimage


In [13]:
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
# dropout
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import Input, Dense, Reshape, Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, Activation, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.datasets import mnist

# Hyperparameters
z_dim = 100  # Latent vector size
img_shape = (64, 64, 3)  # Image shape
batch_size = 64
epochs = 10000
lr = 0.0002
beta_1 = 0.5

# Load dataset (we'll use a resized version of the MNIST dataset for simplicity)
(X_train, _), (_, _) = mnist.load_data()
X_train = X_train / 127.5 - 1.0  # Normalize to [-1, 1]
X_train = np.expand_dims(X_train, axis=-1)
X_train = np.repeat(X_train, 3, axis=-1)  # Convert to RGB images
X_train = np.array([np.resize(img, (64, 64, 3)) for img in X_train])
# Build the generator
def generator(z_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=z_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))  # 64x64 images instead of 128x128
    return model


# Build the discriminator
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    return model


# Relativistic average least squares loss functions
def relativistic_loss(real, fake):
    return K.mean(K.pow(real - fake - 1, 2), axis=0) + K.mean(K.pow(fake - real + 1, 2), axis=0)

# Compile the models
def compile_models(generator, discriminator):
    # Build the discriminator
    discriminator.compile(loss=relativistic_loss, optimizer=Adam(learning_rate=lr, beta_1=beta_1), metrics=['accuracy'])

    # Build the GAN
    z = Input(shape=(z_dim,))
    img = generator(z)
    discriminator.trainable = False
    validity = discriminator(img)
    gan = Model(z, validity)
    gan.compile(loss=relativistic_loss, optimizer=Adam(learning_rate=lr, beta_1=beta_1))

    return gan


# Create the models
generator = build_generator(z_dim)
discriminator = build_discriminator(img_shape)

# Compile the models
gan = compile_models(generator, discriminator)

# Training loop
def train(generator, discriminator, gan, epochs, batch_size, z_dim):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        # Select a random half-batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_images = X_train[idx]

        # Generate a half-batch of fake images
        noise = np.random.normal(0, 1, (half_batch, z_dim))
        fake_images = generator.predict(noise)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(real_images, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # Print the progress
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}] [G loss: {g_loss}]")

        # Save generated images at intervals
        if epoch % 1000 == 0:
            save_imgs(epoch, generator)

# Function to save generated images
def save_imgs(epoch, generator):
    noise = np.random.normal(0, 1, (25, z_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images to [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Plot the images
    fig, axs = plt.subplots(5, 5)
    cnt = 0
    for i in range(5):
        for j in range(5):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"images_{epoch}.png")
    plt.close()

# Start training
train(generator, discriminator, gan, epochs, batch_size, z_dim)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_26 (LeakyReLU)      │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d (ZeroPadding2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 17, 17, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_27 (LeakyReLU)      │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 9, 9, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 9, 9, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_28 (LeakyReLU)      │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 5, 5, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 5, 5, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_29 (LeakyReLU)      │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │         6,401 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 396,609 (1.51 MB)

 Trainable params: 395,713 (1.51 MB)

 Non-trainable params: 896 (3.50 KB)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "functional_31" is incompatible with the layer: expected shape=(None, 64, 64, 3), found shape=(None, 128, 128, 3)[0m

Arguments received by Sequential.call():
  • args=('<KerasTensor shape=(None, 128, 128, 3), dtype=float32, sparse=False, name=keras_tensor_290>',)
  • kwargs={'mask': 'None'}